<a href="https://colab.research.google.com/github/Tanzir11/Image_Caption_Generator/blob/main/Captioon_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive

Mounted at /content/gdrive
/content/gdrive/MyDrive


In [4]:
!mkdir "Caption_Gen"

In [5]:
%cd Caption_Gen

/content/gdrive/MyDrive/Caption_Gen


In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.2 MB/s eta 0:00:00


In [8]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

In [9]:
link = "nlpconnect/vit-gpt2-image-captioning"

In [12]:
# Creating the instance for the model
model = VisionEncoderDecoderModel.from_pretrained(link)

featching the feature extractor as well as tokenizer

In [13]:
from transformers.models.vit.feature_extraction_vit import ViTFeatureExtractor
feature_extractor = ViTFeatureExtractor.from_pretrained(link)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [14]:
tokenizer = AutoTokenizer.from_pretrained(link)

### Checking if we have GPU or not

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [18]:
# introducing some variables
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

In [20]:
pwd

'/content/gdrive/MyDrive/Caption_Gen'

In [25]:
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds


predict_step(["Image1 (1).png"])

['a man kicking a soccer ball on a field']

Let's make some changes

In [33]:
import os
import csv
def predict_image_in_dir(dir_path, output_file):
  images = []
  for filename in os.listdir(dir_path):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
      image_path = os.path.join(dir_path, filename)
      i_image = Image.open(image_path)
      if i_image.mode != "RGB":
        i_image = i_image.convert(mode="RGB")
    images.append((filename, i_image))


  pixel_values = feature_extractor(images=[x[1] for x in images], return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)
  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]

  result = []
  for i in range(len(images)):
    result.append([images[i][0], preds[i]])

  with open(output_file, 'w', newline="") as file:
      writer = csv.writer(file)
      writer.writerow(["Filename", "Caption"])
      writer.writerows(result)
    
  return

In [34]:
predict_image_in_dir("/content/gdrive/MyDrive/Caption_Gen", "Caption")